In [1]:
import os
import json
import pandas as pd
import torch
import numpy as np
from sklearn.model_selection import train_test_split
from transformers import (
    AutoTokenizer, 
    AutoModelForSequenceClassification, 
    TrainingArguments, 
    Trainer
)
from torch.utils.data import Dataset
import multimolecule
# --- IMPORT MODULE CỦA BẠN ---
from metrics import compute_metrics

# --- CẤU HÌNH ĐƯỜNG DẪN (Bạn sửa ở đây) ---
BASE_PATH = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT"
DATA_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data"      # Folder chứa data input
RESULT_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\result"                      # Folder lưu kết quả JSON
OUTPUT_MODEL_DIR = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\pretrained_model"             # Folder chứa các model .h5

# Sửa dòng này:
BASE_MODEL_NAME = "multimolecule/splicebert-human.510"

# Training Config
BATCH_SIZE = 16       
GRAD_ACCUMULATION = 1 
EPOCHS = 3           
LEARNING_RATE = 2e-5

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Training on: {DEVICE}")

# Class hỗ trợ lưu JSON không bị lỗi Numpy
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer): return int(obj)
        if isinstance(obj, np.floating): return float(obj)
        if isinstance(obj, np.ndarray): return obj.tolist()
        return super(NpEncoder, self).default(obj)

🚀 Training on: cuda


In [2]:
class SpliceTrainDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

def prepare_dataset(file_path, tokenizer):
    """Đọc file CSV, chia train/val và tokenize"""
    df = pd.read_csv(file_path)
    sequences = df['sequence'].tolist()
    labels = df['Splicing_types'].tolist()
    
    # Chia 80% Train - 20% Val
    train_seqs, val_seqs, train_labels, val_labels = train_test_split(
        sequences, labels, test_size=0.2, random_state=42, stratify=labels
    )
    
    train_encodings = tokenizer(train_seqs, truncation=True, padding=True, max_length=512)
    val_encodings = tokenizer(val_seqs, truncation=True, padding=True, max_length=512)
    
    return SpliceTrainDataset(train_encodings, train_labels), SpliceTrainDataset(val_encodings, val_labels)

In [3]:
def compute_metrics_wrapper(eval_pred):
    """
    Chuyển đổi output của Trainer (Logits) thành input cho metrics.py (Probs)
    """
    logits, labels = eval_pred
    
    # Trainer trả về numpy array, cần dùng softmax để ra xác suất
    # Lưu ý: logits có thể là tuple, lấy phần tử đầu tiên nếu cần
    if isinstance(logits, tuple):
        logits = logits[0]
        
    # Chuyển logits -> probabilities (Softmax)
    probs = torch.nn.functional.softmax(torch.tensor(logits), dim=-1).numpy()
    preds = np.argmax(probs, axis=-1)
    
    # Gọi hàm từ file metrics.py của bạn
    # Hàm này trả về dict {'accuracy':..., 'f1':..., 'auc':...}
    return compute_metrics(labels, preds, probs)

In [4]:
def run_training(data_filename, output_subfolder_name):
    print(f"\n{'='*20} TRAINING: {output_subfolder_name} {'='*20}")
    
    input_file = os.path.join(DATA_FOLDER, data_filename)
    save_model_path = os.path.join(OUTPUT_MODEL_DIR, output_subfolder_name)
    
    if not os.path.exists(input_file):
        print(f"❌ Không tìm thấy data: {input_file}")
        return

    # 1. Load Tokenizer & Model
    print(f"⏳ Loading {BASE_MODEL_NAME}...")
    try:
        # Load Tokenizer
        tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL_NAME)
        
        # Load Model
        model = AutoModelForSequenceClassification.from_pretrained(
            BASE_MODEL_NAME, 
            num_labels=3,
            trust_remote_code=True
        )
        model.to(DEVICE)
    except Exception as e:
        print(f"❌ Lỗi tải model: {e}")
        return
    
    # 2. Prepare Data
    print("⏳ Đang chuẩn bị dữ liệu...")
    try:
        train_dataset, val_dataset = prepare_dataset(input_file, tokenizer)
    except Exception as e:
        print(f"❌ Lỗi xử lý dữ liệu: {e}")
        return
    
    # 3. Setup Trainer
    # --- SỬA LỖI TẠI ĐÂY: Thay evaluation_strategy bằng eval_strategy ---
    training_args = TrainingArguments(
        output_dir=f"./checkpoints/{output_subfolder_name}",
        num_train_epochs=EPOCHS,
        per_device_train_batch_size=BATCH_SIZE,
        per_device_eval_batch_size=BATCH_SIZE,
        gradient_accumulation_steps=GRAD_ACCUMULATION,
        fp16=False,               
        bf16=True,                  
        tf32=True,
        warmup_steps=500,
        weight_decay=0.01,
        logging_dir='./logs',
        logging_steps=50,
        eval_strategy="epoch",        # <--- DÒNG ĐÃ SỬA (Phiên bản mới dùng eval_strategy)
        save_strategy="epoch",        
        load_best_model_at_end=True,  
        metric_for_best_model="eval_loss",
        learning_rate=LEARNING_RATE,
        dataloader_num_workers=0 
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics_wrapper, 
    )
    
    # 4. Start Training
    print("🚀 Bắt đầu Train...")
    trainer.train()
    
    # 5. Lưu Model Final
    print(f"💾 Đang lưu model vào: {save_model_path}")
    model.save_pretrained(save_model_path)
    tokenizer.save_pretrained(save_model_path)
    
    # 6. Đánh giá & Ghi JSON
    print("📊 Đang tính toán metrics cuối cùng...")
    final_metrics = trainer.evaluate()
    
    output_json_path = os.path.join(RESULT_FOLDER, f"train_result_{output_subfolder_name}.json")
    
    final_output = {
        "meta_data": {
            "task": "training_validation",
            "model_name": output_subfolder_name,
            "source_data": data_filename,
            "base_model": BASE_MODEL_NAME
        },
        "metrics": final_metrics
    }
    
    os.makedirs(RESULT_FOLDER, exist_ok=True)
    with open(output_json_path, 'w') as f:
        json.dump(final_output, f, cls=NpEncoder, indent=4)
        
    print(f"✅ Đã lưu kết quả metrics vào: {output_json_path}")

In [5]:
if __name__ == "__main__":
    # Danh sách các cặp (File Data, Tên Model tương ứng trong folder pretrained_model)
    # Ví dụ: Data test tỉ lệ 1:1:1 chạy với model train trên tỉ lệ 1:1:1
    
    tasks = [
        # ("Tên_file_data.csv", "Tên_folder_hoặc_file_model")
        ("test_1_1_1.csv", "SpliceBERT_ratio_1_1_1"), 
        ("test_2_1_1.csv", "SpliceBERT_ratio_2_1_1"),
        ("test_4_1_1.csv", "SpliceBERT_ratio_4_1_1"),
        ("test_10_1_1.csv", "SpliceBERT_ratio_10_1_1"),
        ("test_data.csv", "SpliceBERT_ratio_100_1_1")
    ]

    print("🚀 BẮT ĐẦU QUÁ TRÌNH KIỂM THỬ...")
    
    for data_file, model_name in tasks:
        run_training(data_file, model_name)
        
    print("\n🏁 HOÀN TẤT TOÀN BỘ!")

🚀 BẮT ĐẦU QUÁ TRÌNH KIỂM THỬ...

==================== TRAINING: SpliceBERT_ratio_1_1_1 ====================
⏳ Loading multimolecule/splicebert-human.510...


Loading weights:   0%|          | 0/101 [00:00<?, ?it/s]

SpliceBertForSequencePrediction LOAD REPORT from: multimolecule/splicebert-human.510
Key                                 | Status     | 
------------------------------------+------------+-
lm_head.transform.dense.weight      | UNEXPECTED | 
lm_head.transform.layer_norm.weight | UNEXPECTED | 
lm_head.decoder.weight              | UNEXPECTED | 
lm_head.transform.dense.bias        | UNEXPECTED | 
lm_head.transform.layer_norm.bias   | UNEXPECTED | 
lm_head.bias                        | UNEXPECTED | 
sequence_head.decoder.weight        | MISSING    | 
model.pooler.dense.bias             | MISSING    | 
model.pooler.dense.weight           | MISSING    | 
sequence_head.decoder.bias          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


⏳ Đang chuẩn bị dữ liệu...


`logging_dir` is deprecated and will be removed in v5.2. Please set `TENSORBOARD_LOGGING_DIR` instead.


🚀 Bắt đầu Train...


c:\Users\Dung\anaconda3\envs\splice_env\lib\site-packages\multimolecule\modules\criterions\generic.py:61: UserWarning: `problem_type` is not set. Assuming multiclass. 
This can lead to unexpected behavior. Please set `problem_type` explicitly.
  warn(


Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [1]:
import os
import json
import time
import pandas as pd
import torch
import numpy as np
import multimolecule  # Import để nhận diện SpliceBERT
from torch.utils.data import Dataset, DataLoader
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from tqdm.notebook import tqdm
from sklearn.metrics import confusion_matrix

# --- Import file metrics của bạn ---
# Đảm bảo file metrics.py nằm cùng thư mục với file notebook này
from metrics import compute_metrics

# ==============================================================================
# 1. CẤU HÌNH (BẠN CHỈ CẦN SỬA Ở ĐÂY)
# ==============================================================================

# Đường dẫn folder chứa các file CSV đã xử lý (đã crop 510bp)
DATA_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\data"

# Đường dẫn folder để lưu kết quả JSON
RESULT_FOLDER = r"D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\result"

# Đường dẫn Model bạn muốn test
# - Nếu là model trên mạng: để tên (vd: "multimolecule/splicebert-human.510")
# - Nếu là model trong máy: điền đường dẫn folder (vd: r"D:\Study\...\pretrained_model\SpliceBERT_ratio_1_1_1")
MODEL_PATH = "multimolecule/splicebert-human.510"  

BIAS_ADJUSTMENT = None

# Cấu hình phần cứng cho RTX 5080
BATCH_SIZE = 256        # Inference rất nhẹ, 5080 có thể gánh 256-512 mẫu/lần
USE_BF16 = True         # Bật BFloat16 (Nhanh & Chuẩn trên RTX 50-series)
COMPILE_MODEL = False    # Bật torch.compile (Tăng tốc cực mạnh trên PyTorch 2.0+)

# ==============================================================================

# Setup thiết bị
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"🚀 Hardware: {torch.cuda.get_device_name(0)} | CUDA: {torch.version.cuda}")

# Class Dataset (Giản lược cho Inference)
class SpliceInferenceDataset(Dataset):
    def __init__(self, csv_file, tokenizer, max_len=512):
        self.df = pd.read_csv(csv_file)
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        # Giả định data đã crop chuẩn 510bp
        seq = str(self.df.iloc[idx]['sequence'])
        label = int(self.df.iloc[idx]['Splicing_types'])
        
        encoding = self.tokenizer(
            seq,
            add_special_tokens=True,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )

        return {
            'input_ids': encoding['input_ids'].flatten(),
            'attention_mask': encoding['attention_mask'].flatten(),
            'labels': torch.tensor(label, dtype=torch.long)
        }

# Class hỗ trợ lưu JSON
class NpEncoder(json.JSONEncoder):
    def default(self, obj):
        if isinstance(obj, np.integer): return int(obj)
        if isinstance(obj, np.floating): return float(obj)
        if isinstance(obj, np.ndarray): return obj.tolist()
        return super(NpEncoder, self).default(obj)

def run_inference(data_filename):
    print(f"\n{'='*20} PROCESSING: {data_filename} {'='*20}")
    
    file_path = os.path.join(DATA_FOLDER, data_filename)
    if not os.path.exists(file_path):
        print(f"❌ Không tìm thấy file: {file_path}")
        return

    # 1. Load Model & Tokenizer
    print(f"⏳ Loading Model: {MODEL_PATH}")
    try:
        tokenizer = AutoTokenizer.from_pretrained(MODEL_PATH, trust_remote_code=True)
        model = AutoModelForSequenceClassification.from_pretrained(
            MODEL_PATH, 
            num_labels=3,
            trust_remote_code=True
        )
        model.to(DEVICE)
        model.eval() # Chế độ đánh giá (không train)
        
        # Tối ưu hóa Model (Optional)
        if COMPILE_MODEL:
            print("🚀 Compiling model for RTX 5080...")
            try:
                model = torch.compile(model)
            except Exception as e:
                print(f"⚠️ Compile thất bại (vẫn chạy bình thường): {e}")
                
    except Exception as e:
        print(f"❌ Lỗi load model: {e}")
        return

    # 2. Prepare Data
    dataset = SpliceInferenceDataset(file_path, tokenizer)
    # num_workers=0 để tránh lỗi trên Windows
    loader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

    # 3. Inference Loop
    all_preds = []
    all_labels = []
    all_probs = []
    
    print(f"🚀 Đang chạy dự đoán trên {len(dataset)} mẫu...")
    start_time = time.time()
    
    # Dùng torch.autocast để chạy BF16 (nhanh hơn FP32)
    dtype = torch.bfloat16 if USE_BF16 and torch.cuda.is_bf16_supported() else torch.float32
    
    with torch.no_grad(): # Tắt tính toán đạo hàm (Tiết kiệm VRAM)
        for batch in tqdm(loader):
            input_ids = batch['input_ids'].to(DEVICE)
            attention_mask = batch['attention_mask'].to(DEVICE)
            labels = batch['labels'].to(DEVICE)

            with torch.autocast(device_type="cuda", dtype=dtype):
                outputs = model(input_ids, attention_mask=attention_mask)
                logits = outputs.logits
            
            probs = torch.softmax(logits, dim=1)
            preds = torch.argmax(probs, dim=1)

            all_preds.extend(preds.cpu().float().numpy())
            all_labels.extend(labels.cpu().float().numpy())
            all_probs.extend(probs.cpu().float().numpy())

    inference_time = time.time() - start_time
    print(f"⏱️ Thời gian: {inference_time:.2f}s ({len(dataset)/inference_time:.0f} seq/s)")

# 4. Tính Metrics & Confusion Matrix
    print("📊 Đang tính metrics...")
    try:
        # Tính các chỉ số cơ bản (Accuracy, F1, AUC...)
        metric_results = compute_metrics(all_labels, all_preds, probs=np.array(all_probs))
        
        # Tính Confusion Matrix
        cm = confusion_matrix(all_labels, all_preds)
        
        # Xử lý Ratio từ tên file (Ví dụ: test_1_1_1.csv -> 1-1-1)
        # Logic: Bỏ đuôi .csv, bỏ tiền tố "test_", thay _ bằng -
        ratio_str = data_filename.replace(".csv", "").replace("test_", "").replace("_", "-")
        
        # Tạo cấu trúc JSON phẳng như yêu cầu
        final_output = metric_results.copy() # Copy các metrics vào level ngoài cùng
        
        final_output["confusion_matrix"] = cm.tolist() # Thêm ma trận nhầm lẫn
        final_output["meta"] = {
            "bias_applied": BIAS_ADJUSTMENT if BIAS_ADJUSTMENT else [0.0, 0.0, 0.0],
            "ratio": ratio_str
        }
        
    except Exception as e:
        print(f"⚠️ Lỗi tính metrics: {e}")
        return

    # 5. Lưu kết quả
    model_name_clean = os.path.basename(MODEL_PATH).replace(":", "_").replace("/", "_")
    output_filename = f"result_{data_filename.replace('.csv', '')}_on_{model_name_clean}.json"
    output_path = os.path.join(RESULT_FOLDER, output_filename)
    
    os.makedirs(RESULT_FOLDER, exist_ok=True)
    with open(output_path, 'w') as f:
        json.dump(final_output, f, cls=NpEncoder, indent=4)
        
    print(f"✅ Đã lưu kết quả chuẩn format tại: {output_path}")
    print(f"   Accuracy: {final_output.get('accuracy', 0):.4f}")

# ==============================================================================
# CHẠY CHƯƠNG TRÌNH
# ==============================================================================
if __name__ == "__main__":
    # Danh sách các file cần chạy
    files_to_test = [
        "test_1_1_1.csv", 
        "test_2_1_1.csv",
        "test_4_1_1.csv",
        "test_10_1_1.csv",
        "test_data.csv" # File mới của bạn
    ]
    
    print("🎬 BẮT ĐẦU CHẠY INFERENCE HÀNG LOẠT...")
    for f in files_to_test:
        run_inference(f)
    print("\n🏁 HOÀN TẤT!")

🚀 Hardware: NVIDIA GeForce RTX 5080 | CUDA: 12.8
🎬 BẮT ĐẦU CHẠY INFERENCE HÀNG LOẠT...

==================== PROCESSING: test_1_1_1.csv ====================
⏳ Loading Model: multimolecule/splicebert-human.510


Loading weights:   0%|          | 0/101 [00:00<?, ?it/s]

SpliceBertForSequencePrediction LOAD REPORT from: multimolecule/splicebert-human.510
Key                                 | Status     | 
------------------------------------+------------+-
lm_head.transform.layer_norm.bias   | UNEXPECTED | 
lm_head.transform.layer_norm.weight | UNEXPECTED | 
lm_head.decoder.weight              | UNEXPECTED | 
lm_head.bias                        | UNEXPECTED | 
lm_head.transform.dense.bias        | UNEXPECTED | 
lm_head.transform.dense.weight      | UNEXPECTED | 
model.pooler.dense.weight           | MISSING    | 
model.pooler.dense.bias             | MISSING    | 
sequence_head.decoder.weight        | MISSING    | 
sequence_head.decoder.bias          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


🚀 Đang chạy dự đoán trên 26310 mẫu...


  0%|          | 0/103 [00:00<?, ?it/s]

...

Loading weights:   0%|          | 0/101 [00:00<?, ?it/s]

SpliceBertForSequencePrediction LOAD REPORT from: multimolecule/splicebert-human.510
Key                                 | Status     | 
------------------------------------+------------+-
lm_head.transform.layer_norm.bias   | UNEXPECTED | 
lm_head.transform.layer_norm.weight | UNEXPECTED | 
lm_head.decoder.weight              | UNEXPECTED | 
lm_head.bias                        | UNEXPECTED | 
lm_head.transform.dense.bias        | UNEXPECTED | 
lm_head.transform.dense.weight      | UNEXPECTED | 
model.pooler.dense.weight           | MISSING    | 
model.pooler.dense.bias             | MISSING    | 
sequence_head.decoder.weight        | MISSING    | 
sequence_head.decoder.bias          | MISSING    | 

Notes:
- UNEXPECTED	:can be ignored when loading from different task/architecture; not ok if you expect identical arch.
- MISSING	:those params were newly initialized because missing from the checkpoint. Consider training on your downstream task.


🚀 Đang chạy dự đoán trên 938297 mẫu...


  0%|          | 0/3666 [00:00<?, ?it/s]

⏱️ Thời gian: 3047.05s (308 seq/s)
📊 Đang tính metrics...
✅ Đã lưu kết quả chuẩn format tại: D:\Bio_sequence_Research_AITALAB\benchmark\task1_splicing_prediction\SpliceBERT\result\result_test_data_on_splicebert-human.510.json
   Accuracy: 0.0120

🏁 HOÀN TẤT!
